# Gemma3-1B Reasoning Model - Colab/Kaggle Example

This notebook demonstrates how to use the Gemma3-1B reasoning model in Google Colab or Kaggle environments.

The project uses `etils[ecolab]` for automatic environment detection and configuration.

## 1. Setup Environment

In [ ]:
# Clone the repository (if not already cloned)
!git clone https://github.com/YOUR_USERNAME/ee596-fp.git
%cd ee596-fp

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

## 2. Configure Notebook Environment with ecolab

The `setup_notebook_env()` function automatically:
- Detects if you're running in Colab or Kaggle
- Configures plotting for better visualization
- Sets up environment-specific optimizations

In [ ]:
from src import setup_notebook_env, get_device

# Setup notebook environment
setup_notebook_env()

# Detect best device
device = get_device()
print(f"Using device: {device}")

## 3. Authenticate with HuggingFace

Gemma is a gated model. You need to:
1. Accept the license at [google/gemma-3-1b-it](https://huggingface.co/google/gemma-3-1b-it)
2. Get your token from [HuggingFace Settings](https://huggingface.co/settings/tokens)
3. Login below:

In [ ]:
from huggingface_hub import notebook_login

# Login to HuggingFace
notebook_login()

## 4. Load the Model

The model will automatically use GPU if available in Colab/Kaggle.

In [ ]:
from src import load_model

# Load model (base model without fine-tuning)
# For GPU environments, you can use 4-bit quantization to save memory
model = load_model(
    checkpoint_path=None,  # Set to path of fine-tuned weights if you have them
    device="auto",
    load_in_4bit=True if device == "cuda" else False  # Use quantization on GPU
)

print("✅ Model loaded successfully!")

## 5. Test the Model

Let's test the model with a simple math problem.

In [ ]:
# Example question
question = "A store sells apples for $2 each and oranges for $3 each. If Sarah buys 4 apples and 5 oranges, how much does she spend in total?"

# Generate answer
result = model.solve(question, temperature=0.7)

# Display results
print("="*70)
print("QUESTION:")
print(question)
print("\n" + "="*70)
print("REASONING:")
print(result["reasoning"])
print("\n" + "="*70)
print("ANSWER:")
print(result["answer"])
print("="*70)

## 6. Interactive Question Answering

Try your own questions!

In [ ]:
def answer_question(question: str):
    """Helper function to answer a question and display results."""
    result = model.solve(question, temperature=0.7)
    
    print("="*70)
    print("📝 REASONING:")
    print(result["reasoning"] or "(No reasoning found)")
    print("\n✅ ANSWER:")
    print(result["answer"] or "(No answer found)")
    print("="*70)
    
    return result

# Try different questions
questions = [
    "If a train travels at 60 mph for 2.5 hours, how far does it travel?",
    "Why does ice float on water?",
    "What is 15% of 200?"
]

for i, q in enumerate(questions, 1):
    print(f"\n\n### Question {i}: {q}")
    answer_question(q)

## 7. Environment Information

Check what environment we're running in:

In [ ]:
try:
    from etils import ecolab
    
    print("Environment Information:")
    print(f"- Is Notebook: {ecolab.is_notebook()}")
    
    if ecolab.is_notebook():
        print(f"- Notebook Name: {ecolab.get_notebook_name()}")
    
    if hasattr(ecolab, 'is_colab') and ecolab.is_colab():
        print("- Platform: Google Colab")
    elif hasattr(ecolab, 'is_kaggle') and ecolab.is_kaggle():
        print("- Platform: Kaggle")
    else:
        print("- Platform: Other notebook environment")
        
except ImportError:
    print("ecolab not available")

## 8. Batch Processing

Process multiple questions at once:

In [ ]:
import pandas as pd

# List of questions
test_questions = [
    "What is 25 * 4?",
    "How many days are in a leap year?",
    "If a rectangle has length 10 and width 5, what is its area?",
]

# Process all questions
results = []
for q in test_questions:
    result = model.solve(q, temperature=0.3)  # Lower temperature for more deterministic answers
    results.append({
        "Question": q,
        "Answer": result["answer"],
        "Reasoning": result["reasoning"][:100] + "..." if len(result["reasoning"]) > 100 else result["reasoning"]
    })

# Display as DataFrame
df = pd.DataFrame(results)
display(df)

## Next Steps

- Try more complex reasoning problems
- Fine-tune the model on your own dataset
- Evaluate on standard benchmarks like GSM8K
- Experiment with different temperature and sampling settings

For more information, see the [main README](../README.md).